# Make depth summary figures

This notebook is to produce some depth summary figures for the main HELP paper

We use the full sky depth map produced in dmu32/dmu32_AllSky/



In [16]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))


This notebook was run with herschelhelp_internal version: 
1407877 (Mon Feb 4 12:56:29 2019 +0000)


In [17]:
import pyvo as vo
import glob
import time
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import random

import herschelhelp as hh
from herschelhelp_internal.utils import flux_to_mag

from astropy.table import Table, Column, vstack, join, unique

from pymoc import MOC

import pandas as pd

import yaml

from pcigale.sed import SED
from pcigale.sed_modules import get_module

In [ ]:
depths = Table.read('../../../dmu32/dmu32_AllSky/data/')[
    'ferr_ap_u_mean',
    'ferr_ap_g_mean',
    'ferr_ap_r_mean',
    'ferr_ap_i_mean',
    'ferr_ap_z_mean',
    'ferr_ap_y_mean',
    'ferr_ap_j_mean',
    'ferr_ap_h_mean',
    'ferr_ap_k_mean',
    'ferr_ap_ks_mean',
    'ferr_ap_irac_i1_mean',
    'ferr_ap_irac_i2_mean',
    'ferr_ap_irac_i3_mean',
    'ferr_ap_irac_i4_mean',
    'ferr_mips_24_mean',
    'ferr_pacs_green',
    'ferr_pacs_red',
    'ferr_spire_250',
    'ferr_spire_350',
    'ferr_spire_500'
]

## Plot general overview of depths

In [ ]:
bands

In [ ]:
bands_fir = ['u', 'g', 'r', 'i', 'z', 'y', 'J', 'H', 'K', 'Ks', 'i1', 'i2', 'i3', 'i4', 
             '24', '100', '160', '250', '350', '500']

In [ ]:
#s.add_contribution("HELP_SED", orig_spec['WAVE'], orig_spec['LUMIN'])

In [ ]:
cigale_filternames = {
    'u':  'u_prime',
    'g':  'MCam_g',
    'r':  'MCam_r',
    'i':  'MCam_i',
    'z':  'MCam_z',
    'y':  'WFCAM_Y',
    'J':  'WFCAM_J',
    'H':  'WFI_H',
    'K':  'WFI_K',
    'Ks': 'Ks_2mass',
    'i1': 'IRAC1',
    'i2': 'IRAC2',
    'i3': 'IRAC3',
    'i4': 'IRAC4'
}

In [ ]:



gal1 = './data/HELP_J095946.083p021914.438_best_model.fits'
gal2 = './data/HELP_J100130.443p020929.494_best_model.fits'
gal3 = './data/HELP_J095809.302p013203.775_best_model.fits'
gal4 = './data/HELP_J095822.986p013145.336_best_model.figs'
gal5 = './data/HELP_J003412.527-441056.846_best_model.fits'  #z=4
gal6 = './data/HELP_J095738.934+021508.530_best_model.fits' #From hedam - high fluxes
gal7 = './data/HELP_J095818.598+013057.910_best_model.fits'
gal8 = './data/HELP_J095809.488+013225.513_best_model.fits'
gal9 = './data/HELP_J100108.952+022730.528_best_model.fits'
orig_spec = Table.read(gal9)
        
s = SED()
    # This is wrong because the best SED we get from CIGALE is redshifted (written by Yannick)
s.add_contribution("HELP_SED", orig_spec['wavelength'], orig_spec['L_lambda_total'])
    
#z=1
zs = np.arange(1, 5, 1)




In [ ]:
gal_fluxes = np.full([len(zs), len(bands)], np.nan)

In [ ]:
fig, ax = plt.subplots()
for n, z in enumerate(zs):
    print('z = {}:'.format(z))
    sed = s.copy()
    mod = get_module("redshifting", redshift=z)
    mod.process(sed)
    ax.plot(np.log10(sed.wavelength_grid), np.log10(sed.fnu * 1.e-3), label=str(z))
#ax.plot(np.log10(orig_spec['wavelength']), np.log10(orig_spec['Fnu']))

plt.legend()
ax.set_ylim(-7, 0)
#ax.set_xlim(0, 14.75)

In [ ]:
pos = [3561.,     #u  (SDSS)
       4866.,     #g  (GPC1)
       6215.,     #r  (GPC1)
       7545.,     #i  (GPC1)
       8680.,     #z  (GPC1)
       9633.,     #y  (GPC1)
       12510.,    #J  (UKIRT)
       16377.,    #H  (UKIRT)
       22081.,    #K   (UKIRT)
       21496.,    #Ks (WIRCam)
       36000.,    #i1
       45000.,    #i2 
       56000.,    #i3
       80000.,    #i4
       240000.,    #mips_24
       1000000.,    #pacs_100
       1600000.,    #pacs_160
       2500000.,    #spire_250
       3500000.,    #spire_350
       5000000.,    #spire_500
      ]   
fwhms = [
       [3048., 4028.],     #u  (SDSS)
       [3943., 5593.],     #g  (GPC1)
       [5386., 7036.],     #r  (GPC1)
       [6778., 8304.],     #i  (GPC1)
       [8028., 9346.],     #z  (GPC1)
       [9100., 10838.],     #y  (GPC1)
       [11690., 13280],     #J  (UKIRT)
       [14920., 17840.],     #H  (UKIRT)
       [20290., 23800.],     #K   (UKIRT)
       [19578., 23431.],     #Ks (WIRCam)
       [31296, 39614 ],     #i1
       [39173, 50561],      #i2 
       [48983, 65089],     #i3
       [62994, 95876],      #i4
       [240000.,340000.],    #mips_24 GUESSES!!!!
       [1000000.,1500000.],    #pacs_100
       [1600000.,1900000.],    #pacs_160
       [2500000.,3000000.],    #spire_250
       [3500000.,4000000.],    #spire_350
       [5000000.,5500000.]    #spire_500
        ]

In [ ]:
['${}$'.format(band).replace(
    '$Ks$', '').replace(
    'K', 'K/Ks').replace(
     '100', 'PACS 100/160').replace(
     '160', ' ').replace(
     '250', 'SPIRE 250/350/500').replace(
     '350', ' ').replace(
     '500', ' ') for band in bands_fir]

In [ ]:
# fake data
fs = 10  # fontsize
# Angstroms     


#data = np.array([np.array(coverage['ferr_ap_{}_mean_min'.format(b)]) for b in bands]).T

cmap = mpl.cm.viridis
norm = mpl.colors.Normalize(vmin=np.min(areas), vmax=np.max(areas))
scalmap = mpl.cm.ScalarMappable( cmap=cmap, norm=norm)
colors = scalmap.to_rgba(areas)   # The color is the angle
colors[:, 3] = 1

fig, ax = plt.subplots()

ax2 = ax.twiny()

#ax2.set_xlim(3000., 100000)
#ax2.set_xticks(pos)
#ax2.set_xticklabels(['$' + band + '$' for band in bands])
#ax2.set_xlabel('band')
#ax2.set_xscale('log')
ax2.set_xlim(np.log10(3000.), np.log10(10000000))
ax2.set_xticks(np.log10(pos))
ax2.set_xticklabels(['$u$',
 '$g$',
 '$r$',
 '$i$',
 '$z$',
 '$y$',
 '$J$',
 '$H$',
 '  $K/Ks$',
 '',
 '$i1$',
 '$i2$',
 '$i3$',
 '$i4$',
 'MIPS 24',
 '       PACS',
 '',
 '           SPIRE',
 '',
 ''])
ax2.set_xlabel('band')
#ax2.set_xscale('log')


line_styles = [':', '-.', '--', '-']
colours = ['y', 'b', 'g', 'r', 'k']
for n, z in enumerate([0.25, 1, 2, 3, 4]):
    sed = s.copy()
    mod = get_module("redshifting", redshift=z)
    mod.process(sed)
    ax.plot(sed.wavelength_grid*10, 
            np.log10(sed.fnu * 1.e-3),
            c='k',
            #c= colours[n],
            #linestyle = line_styles[n],
            label= 'z = {}'.format(z),
            linewidth=1.0,
            alpha=1.
           )
    for m, band in enumerate(cigale_filternames):
        continue
        if m == 0:
            lab = 'z = {}'.format(z)
        else:
            lab=None
        ax.plot([fwhms[m][0], fwhms[m][1]], [np.log10(gal_fluxes[n, m]   )-3, 
                                     np.log10(gal_fluxes[n, m]   )-3], 
               # c='k',
                c= colours[n],
               #linestyle = line_styles[n],
               label=lab,
                alpha=1.0
               )

        




ax.set_ylabel('log10(Depths [Jy] )')

#ax.set_xticks(pos)
#ax.set_xticklabels(['${}$'.format(band.replace('Ks', '').replace('K', 'K/Ks')) for band in bands])
ax.set_xlim(3000., 10000000)
ax.set_xscale('log')    
ax.set_xlabel('Wavelength [nm]')
ax.set_ylim(-7, 0)
#ax.legend(loc=2)


#widths
log_widths = np.ones(len(pos)) * (pos) * .2# * widths
#log_widths = np.ones(len(pos)) * 1000.
#areas/np.max(areas)
parts = ax.violinplot(data, 
                      positions=pos, 
                      widths=log_widths, 
                      showmeans=False, 
                      showmedians=False, #widths=widths,
        showextrema=False)

for n, part in enumerate(parts['bodies']):
    part.set_facecolor(colors[n])
    part.set_alpha(1.0)

cax, _ = mpl.colorbar.make_axes([ax, ax2])
n_ticks = 7
values = np.linspace(0,1200, n_ticks)
ticks = values/np.max(areas)

cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, ticks = ticks)
cax.set_yticklabels([int(d) for d in values])
cbar.set_label('Area [square degrees]')



plt.figtext(0.335, 0.6, 'z=0.25')
plt.figtext(0.38, 0.45, 'z=1')
plt.figtext(0.4, 0.38, 'z=2')
plt.figtext(0.44, 0.35, 'z=3')
plt.figtext(0.45, 0.27, 'z=4')





plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

#fig.suptitle("Violin Plotting Examples")
#fig.subplots_adjust(hspace=0.4)
#plt.ylim(-10,10)
column_width_cm = 8.9
width_cm = 3.0 * column_width_cm
hieght_cm = width_cm / 1.9
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)
plt.savefig('./figs/band_depths_overviews_areaweighted_with_black_seds_wave.pdf', bbox_inches='tight')
plt.savefig('./figs/band_depths_overviews_areaweighted_with_black_seds_wave.png', bbox_inches='tight')

## Summarise the filters

In [ ]:
for band in bands:
    specific_bands = [f for f in filters if f.split('_')[1] == band.lower()]
    print(specific_bands)
    for specific_band in specific_bands:
        if 'ferr_ap_{}_mean'.format(specific_band) not in coverage.colnames:
            specific_bands.remove(specific_band)
    #print(specific_bands)

In [ ]:
filter_info = Table.read(herschelhelp_python_loc + 'documentation/filters.csv')

In [ ]:
has_broad_typical = np.full(len(filter_info), False)
for band in bands:
    has_broad_typical |= filter_info['filter'] == band
    
filter_info[has_broad_typical]

In [ ]:
has_optical = np.full(len(filter_info), False)
optical_bands = ['u', 'g', 'r', 'i', 'z', 'y']
has_near_infrared = np.full(len(filter_info), False)
near_infrared_bands = ['J', 'H', 'K', 'Ks']

for band in bands:
    if band in optical_bands:
        has_optical |= filter_info['filter'] == band
    if band in near_infrared_bands:
        has_near_infrared |= filter_info['filter'] == band

In [ ]:
def get_bands(camera):
    band_list = []
    for b in filter_info['filter'][filter_info['camera'] == camera]:
        #print(b)
        band_list.append( b)
    band_list_ordered_string = ''
    for band in bands:
        if band in band_list:
            band_list_ordered_string += band
    
    return '$' + band_list_ordered_string + '$'



print(get_bands('HyperSuprimeCam'))

In [ ]:
optical_sentence = ''
for camera in np.unique(filter_info['camera'][has_optical]):
    if camera == 'Standard camera response (camera unknown)':
        continue
    row = filter_info[filter_info['camera'] == camera][0] 
    optical_sentence += 'From {} on {} we have the optical bands {}. '.format(row['camera'].replace('The', 'the'), 
                                                             row['telescope'].replace('The', 'the'),
                                                             get_bands(camera))
    
print(optical_sentence)

In [ ]:
near_infrared_sentence = ''
for camera in np.unique(filter_info['camera'][has_near_infrared]):
    if camera == 'Standard camera response (camera unknown)':
        continue
    row = filter_info[filter_info['camera'] == camera][0] 
    near_infrared_sentence += 'From {} on {} we have the optical bands {}. '.format(row['camera'].replace('The', 'the'), 
                                                             row['telescope'].replace('The', 'the'),
                                                             get_bands(camera))
    
print(near_infrared_sentence)

# Make cumulative 1 d depths for MIPS, PACS, SPIRE

In [ ]:
np.array(en1_depths['ferr_mips_24_mean'])[~np.isnan(en1_depths['ferr_mips_24_mean'])]

In [ ]:


fig, ax = plt.subplots()

#good = np.log10(np.array(g_table['ferr_g_min'])*1.e-6) > -20
#good &= np.log10(np.array(g_table['ferr_g_min'])*1.e-6) <20

mips_pix_mag_depths = np.array(np.log10(en1_depths['ferr_mips_24_mean']*1.e-6 ))
cells, fluxes = np.histogram(mips_pix_mag_depths[~np.isnan(mips_pix_mag_depths)], bins=1000)
cells =np.flip(cells)
fluxes = np.flip(fluxes)

ax.plot(fluxes[1:],
         np.cumsum(cells)*200./cells.sum() ,
        drawstyle='steps')

ax.set_xlabel('log10(2$\sigma$ MIPS 24 depth [Jy]) ')
ax.set_ylabel('Cumulative area [deg.$^2$]')
ax.set_xlim([-5,-4.5])
ax.set_ylim([0,210])
#y_vals = ax.get_yticks()
#ax.set_yticklabels([n for n in y_vals])



plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.0 * column_width_cm
hieght_cm = width_cm 
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)


plt.savefig('./figs/mips_cumulative_area_depth.pdf', bbox_inches='tight')
plt.savefig('./figs/mips_cumulative_area_depth.png', bbox_inches='tight')

In [ ]:


fig, ax = plt.subplots()

#good = np.log10(np.array(g_table['ferr_g_min'])*1.e-6) > -20
#good &= np.log10(np.array(g_table['ferr_g_min'])*1.e-6) <20

pacs_pix_mag_depths = np.array(np.log10(en1_depths['ferr_pacs_green_mean']*1.e-6 ))
cells, fluxes = np.histogram(pacs_pix_mag_depths[~np.isnan(pacs_pix_mag_depths)], bins=1000)
cells =np.flip(cells)
fluxes = np.flip(fluxes)

ax.plot(fluxes[1:],
         np.cumsum(cells)*200./cells.sum() ,
        drawstyle='steps')

ax.set_xlabel('log10(2$\sigma$ PACS 100 depth [Jy]) ')
ax.set_ylabel('Cumulative area [deg.$^2$]')
ax.set_xlim([-2.2,-1.5])
ax.set_ylim([0,210])
#y_vals = ax.get_yticks()
#ax.set_yticklabels([n for n in y_vals])



plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.0 * column_width_cm
hieght_cm = width_cm 
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)


plt.savefig('./figs/pacs_cumulative_area_depth.pdf', bbox_inches='tight')
plt.savefig('./figs/pacs_cumulative_area_depth.png', bbox_inches='tight')

In [ ]:


fig, ax = plt.subplots()

#good = np.log10(np.array(g_table['ferr_g_min'])*1.e-6) > -20
#good &= np.log10(np.array(g_table['ferr_g_min'])*1.e-6) <20

spire_pix_mag_depths = np.array(np.log10(en1_depths['ferr_spire_250_mean']*1.e-6 ))
cells, fluxes = np.histogram(spire_pix_mag_depths[~np.isnan(spire_pix_mag_depths)], bins=1000)
cells =np.flip(cells)
fluxes = np.flip(fluxes)

ax.plot(fluxes[1:],
         np.cumsum(cells)*1100./cells.sum() ,
        drawstyle='steps')

ax.set_xlabel('log10(2$\sigma$ SPIRE 250 depth [Jy])')
ax.set_ylabel('Cumulative area [deg.$^2$]')
#ax.set_xlim([-2.2,-1.5])
ax.set_ylim([0,1200])
#y_vals = ax.get_yticks()
#ax.set_yticklabels([n for n in y_vals])



plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.0 * column_width_cm
hieght_cm = width_cm 
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)


plt.savefig('./figs/spire_cumulative_area_depth.pdf', bbox_inches='tight')
plt.savefig('./figs/spire_cumulative_area_depth.png', bbox_inches='tight')

# 2d depth plots

Make a cumulative 2d area that is at least as deep.

In [ ]:
en1_d = Table.read('../../../dmu32/dmu32_ELAIS-N1/data/depths_elais-n1_20171016.fits')

In [ ]:
fig, ax = plt.subplots()
good = ~np.isnan(en1_d['ferr_ap_irac_i1_mean']) & ~np.isnan(en1_d['ferr_mips_24_mean'])
good &= en1_d['ferr_ap_irac_i1_mean'] < 1.
good &= en1_d['ferr_mips_24_mean'] > 12.
good &= en1_d['ferr_mips_24_mean'] < 18.
ax.hist2d(en1_d['ferr_ap_irac_i1_mean'][good],en1_d['ferr_mips_24_mean'][good],  bins=25)


In [ ]:
good &= en1_d['ferr_ap_irac_i1_mean'] < 1.
good &= en1_d['ferr_mips_24_mean'] > 12.
good &= en1_d['ferr_mips_24_mean'] < 18.

hist = np.histogram2d(en1_d['ferr_ap_irac_i1_mean'][good],en1_d['ferr_mips_24_mean'][good],  bins=25)
hist

In [ ]:
vals

In [ ]:
from pymoc import MOC
t_moc = MOC(13, [en1_d['hp_idx_O_13'][0] ])
t_moc.area_sq_deg

In [ ]:
band1 = ['ferr_ap_irac_i1_mean',0.,1.]
band2 = ['ferr_mips_24_mean',0.,20. ]
#good = ~np.isnan(en1_d[band1[0]] ) & ~np.isnan(en1_d[band2[0]] )
good = en1_d[band1[0]] > band1[1]
good &= en1_d[band1[0]] < band1[2]
good &= en1_d[band2[0]] > band2[1]
good &= en1_d[band2[0]] < band2[2]

hist = np.histogram2d(en1_d[band1[0]][good],en1_d[band2[0]][good],  bins=1000)
vals = hist[0].copy() * t_moc.area_sq_deg
#vals = np.flip(np.cumsum(np.cumsum(np.flip(vals), axis=0),axis=1))
vals = np.cumsum(np.cumsum(vals, axis=0),axis=1)
fig, ax = plt.subplots()


#im = imshow(np.flip(Z, axis=0),cmap="viridis", extent=[t1,t2,p1,p2], aspect="auto") # drawing the function
# adding the Contour lines with labels
#cset = contour(Z,arange(-1,1.5,0.2),linewidths=2,cmap=cm.Set2)
#clabel(cset,inline=True,fmt='%1.1f',fontsize=10)
#cbar = fig.colorbar(im) # adding the colobar on the right
#cbar.set_label('Yield [tonnes ha$^{-1}$]')
# latex fashion title
#title('$z=(1-x^2+y^3) e^{-(x^2+y^2)/2}$')

CS = ax.contour(hist[1][:-1] + (hist[1][1]-hist[1][0])/2, 
                hist[2][:-1] + (hist[2][1]-hist[2][0])/2, 
                vals
                #,[levels]
               )
ax.clabel(CS, inline=1, fontsize=10)
ax.set_xlim([0.5, 1.])
ax.set_ylim([12., 19.])

ax.set_xlabel('IRAC $i1$ Depth [$\mu$Jy]')
ax.set_ylabel('MIPS 24$\mu$m Depth [$\mu$Jy]')


plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.0 * column_width_cm
hieght_cm = width_cm 
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)

plt.savefig('./figs/2d_i1_mips_depth.pdf', bbox_inches='tight')
plt.savefig('./figs/2d_i1_mips_depth.png', bbox_inches='tight')

In [ ]:
band1 = ['ferr_ap_r_mean',0.,1.2]
band2 = ['ferr_ap_k_mean',0.,1.1 ]
#good = ~np.isnan(en1_d[band1[0]] ) & ~np.isnan(en1_d[band2[0]] )
good = en1_d[band1[0]] > band1[1]
good &= en1_d[band1[0]] < band1[2]
good &= en1_d[band2[0]] > band2[1]
good &= en1_d[band2[0]] < band2[2]

hist = np.histogram2d(en1_d[band1[0]][good],en1_d[band2[0]][good],  bins=100)
vals = hist[0].copy() * t_moc.area_sq_deg
#vals = np.flip(np.cumsum(np.cumsum(np.flip(vals), axis=0),axis=1))
vals = np.cumsum(np.cumsum(vals, axis=0),axis=1)
fig, ax = plt.subplots()


#im = imshow(np.flip(Z, axis=0),cmap="viridis", extent=[t1,t2,p1,p2], aspect="auto") # drawing the function
# adding the Contour lines with labels
#cset = contour(Z,arange(-1,1.5,0.2),linewidths=2,cmap=cm.Set2)
#clabel(cset,inline=True,fmt='%1.1f',fontsize=10)
#cbar = fig.colorbar(im) # adding the colobar on the right
#cbar.set_label('Yield [tonnes ha$^{-1}$]')
# latex fashion title
#title('$z=(1-x^2+y^3) e^{-(x^2+y^2)/2}$')

CS = ax.contour(hist[1][:-1] + (hist[1][1]-hist[1][0])/2, 
                hist[2][:-1] + (hist[2][1]-hist[2][0])/2, 
                vals
                #,[levels]
               )
ax.clabel(CS, inline=1, fontsize=10)
ax.set_xlim([0.05, .07075])
ax.set_ylim([0.3, 1.1])

ax.set_xlabel('Deepest $r$ Depth [$\mu$Jy]')
ax.set_ylabel('Deepest $K$ or $K_s$ Depth [$\mu$Jy]')

plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.0 * column_width_cm
hieght_cm = width_cm 
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)

plt.savefig('./figs/2d_r_k_depth.pdf', bbox_inches='tight')
plt.savefig('./figs/2d_r_k_depth.png', bbox_inches='tight')

In [ ]:
np.sum(vals)

In [ ]:
band1 = ['ferr_pacs_green_mean',0.,40000.2]
band2 = ['ferr_spire_350_mean',0.,4500.1 ]
#good = ~np.isnan(en1_d[band1[0]] ) & ~np.isnan(en1_d[band2[0]] )
good = en1_d[band1[0]] > band1[1]
good &= en1_d[band1[0]] < band1[2]
good &= en1_d[band2[0]] > band2[1]
good &= en1_d[band2[0]] < band2[2]

hist = np.histogram2d(en1_d[band1[0]][good],en1_d[band2[0]][good],  bins=100)
vals = hist[0].copy() * t_moc.area_sq_deg
#vals = np.flip(np.cumsum(np.cumsum(np.flip(vals), axis=0),axis=1))
vals = np.cumsum(np.cumsum(vals, axis=0),axis=1)
fig, ax = plt.subplots()


#im = imshow(np.flip(Z, axis=0),cmap="viridis", extent=[t1,t2,p1,p2], aspect="auto") # drawing the function
# adding the Contour lines with labels
#cset = contour(Z,arange(-1,1.5,0.2),linewidths=2,cmap=cm.Set2)
#clabel(cset,inline=True,fmt='%1.1f',fontsize=10)
#cbar = fig.colorbar(im) # adding the colobar on the right
#cbar.set_label('Yield [tonnes ha$^{-1}$]')
# latex fashion title
#title('$z=(1-x^2+y^3) e^{-(x^2+y^2)/2}$')

CS = ax.contour(hist[1][:-1] + (hist[1][1]-hist[1][0])/2, 
                hist[2][:-1] + (hist[2][1]-hist[2][0])/2, 
                vals
                #,[levels]
               )
ax.clabel(CS, inline=1, fontsize=10)
ax.set_xlim([9000., 40000.])
ax.set_ylim([2000., 4500.])

ax.set_xlabel('PACS 100$\mu$m Depth [$\mu$Jy]')
ax.set_ylabel('SPIRE 250$\mu$m Depth [$\mu$Jy]')

plt.rc('font', family='serif', serif='Times')
plt.rc('text') #, usetex=True)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)

column_width_cm = 8.9
width_cm = 1.0 * column_width_cm
hieght_cm = width_cm 
width_inches = width_cm/2.5
hieght_inches = hieght_cm/2.5
fig.set_size_inches(width_inches, hieght_inches)

plt.savefig('./figs/2d_pacs_spire_depth.pdf', bbox_inches='tight')
plt.savefig('./figs/2d_pacs_spire_depth.png', bbox_inches='tight')